# **Figure 1**. (a–c) Average clustering coefficient $\langle C \rangle$ and temporal evolution of (d–f) global clustering coefficient $C$, (g–i) small-world index $\omega$, and (j–l) average shortest path length $L$.

In [37]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

# === Style settings ===
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

fontsize_labels = 30
fontsize_ticks = 25
fontsize_titles = 35
fontsize_panel_labels = 30
fontsize_legend = 25
panel_spacing = 0.25
point_size_scatter = 150

# === Color map ===
color_map = {
    "0.100": 'tab:orange', "0.150": 'tab:cyan', "0.200": 'tab:purple', "0.250": 'tab:brown',
    "0.300": 'b', "0.350": 'tab:pink', "0.360": 'tab:olive', "0.370": 'tab:gray',
    "0.400": 'g', "0.450": 'tab:blue', "0.500": 'r'
}

# === Mapping from p to <k> ===
p_to_k = {
    "0.060": 18,
    "0.120": 36,
    "0.180": 54
}

# === Metrics and axis limits ===
dynamic_metrics = ["clustering", "omega", "path_length"]
metric_labels = [r"$\langle C \rangle$", r"$C$", r"$\omega$", r"$L$"]
y_limits = [
    (0.0, 0.9),   # <C>
    (0.0, 0.9),   # C
    (0.0, 10.0),  # omega
    (1.7, 3.2)    # L
]

# === Initialize figure ===
fig, axs = plt.subplots(3, 4, figsize=(30, 20))
panel_labels = [f"({chr(97+i)})" for i in range(12)]
panel_label_matrix = np.array(panel_labels).reshape(4, 3).T

# === Get epsilon values and legend once from the first file
first_folder = next(iter(p_to_k))
df_temp = pd.read_csv(os.path.join(f"results/results_p{first_folder}", "network_metrics.txt"))
epsilons_for_legend = sorted(df_temp["epsilon"].unique())
legend_handles = [
    plt.Line2D([0], [0], color=color_map[f"{eps:.3f}"], lw=2, label=rf"$\varepsilon={eps:.2f}$")
    for eps in epsilons_for_legend
]

# === Loop through each p folder ===
for row_idx, (p_str, k_val) in enumerate(p_to_k.items()):
    folder = f"results/results_p{p_str}"
    path = os.path.join(folder, "network_metrics.txt")
    if not os.path.exists(path):
        continue

    df = pd.read_csv(path)
    df.rename(columns={"<s>": "s_mean", "<s>_std": "s_std"}, inplace=True)
    epsilon_values = sorted(df["epsilon"].unique())
    n_last = 100000

    # Panel for <C>
    ax = axs[row_idx, 0]
    ax.set_ylim(y_limits[0])
    avg_clustering = []
    std_clustering = []

    for eps in epsilon_values:
        df_eps = df[df["epsilon"] == eps]
        max_step = df_eps["time_step"].max()
        df_final = df_eps[df_eps["time_step"] >= max_step - n_last]
        grouped = df_final.groupby("simulation")["clustering"].mean()
        avg_clustering.append(grouped.mean())
        std_clustering.append(grouped.std())

    eps_array = np.array(epsilon_values)
    colors = [color_map.get(f"{eps:.3f}", 'k') for eps in eps_array]
    ax.scatter(eps_array, avg_clustering, c=colors, s=point_size_scatter, zorder=2)
    ax.plot(eps_array, avg_clustering, color='black', linewidth=1.2, zorder=1)
    ax.errorbar(eps_array, avg_clustering, yerr=std_clustering,
                fmt='none', ecolor='black', elinewidth=2.0, capsize=3, capthick=1.5, zorder=3)

    if row_idx == 2:
        ax.set_xlabel(r"$\varepsilon$", fontsize=fontsize_labels)
    if row_idx == 0:
        ax.set_title(metric_labels[0], fontsize=fontsize_titles)

    ax.set_ylabel(rf"$\langle k \rangle = {k_val}$", fontsize=fontsize_labels)
    ax.grid(True)
    ax.tick_params(axis='both', labelsize=fontsize_ticks)
    ax.text(0.02, 0.93, panel_label_matrix[row_idx, 0], transform=ax.transAxes,
            fontsize=fontsize_panel_labels, fontweight='bold', va='top', ha='left')

    # Panels for C, omega, L
    for col_offset, (metric, label, y_lim) in enumerate(zip(dynamic_metrics, metric_labels[1:], y_limits[1:]), start=1):
        ax = axs[row_idx, col_offset]
        ax.set_ylim(y_lim)

        for eps in epsilon_values:
            df_eps = df[df["epsilon"] == eps]
            grouped = df_eps.groupby("time_step")[metric].agg(['mean', 'std']).reset_index()
            eps_str = f"{eps:.3f}"
            color = color_map.get(eps_str, 'k')

            ax.plot(grouped["time_step"], grouped["mean"], color=color, label=rf"$\varepsilon={eps:.2f}$")
            ax.fill_between(grouped["time_step"],
                            grouped["mean"] - grouped["std"],
                            grouped["mean"] + grouped["std"],
                            color=color, alpha=0.2)

        if row_idx == 2:
            ax.set_xlabel("Time steps", fontsize=fontsize_labels)
        if row_idx == 0:
            ax.set_title(label, fontsize=fontsize_titles)

        ax.grid()
        ax.tick_params(axis='both', labelsize=fontsize_ticks)
        ax.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
        ax.ticklabel_format(style='sci', axis='x', scilimits=(0, 0))
        ax.xaxis.get_offset_text().set_fontsize(fontsize_ticks)

        ax.text(0.02, 0.93, panel_label_matrix[row_idx, col_offset], transform=ax.transAxes,
                fontsize=fontsize_panel_labels, fontweight='bold', va='top', ha='left')

# === Global legend (bottom center)
fig.legend(handles=legend_handles,
           loc='upper center',
           bbox_to_anchor=(0.5, -0.01),
           ncol=len(legend_handles),
           fontsize=fontsize_legend,
           frameon=True,
           handlelength=2.0).get_frame().set_edgecolor('black')

# === Save output
output_dir = "plots_paper/fig1"
os.makedirs(output_dir, exist_ok=True)
plt.tight_layout(rect=[0, 0.06, 1, 1])
fig.subplots_adjust(bottom=0.08, wspace=panel_spacing, hspace=panel_spacing)
plt.savefig(os.path.join(output_dir, "fig1.pdf"), bbox_inches="tight")
plt.savefig(os.path.join(output_dir, "fig1.png"), dpi=300, bbox_inches="tight")
plt.close()

print("✅ Final figure saved as plots_paper/fig1/fig1.[pdf|png]")


✅ Final figure saved as plots_paper/fig1/fig1.[pdf|png]


# **Figure 2 and 3**. Community Structure by Louvain Algorithm.

# **Figure 4**. Degree Distribution.

In [34]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

# === Plot style settings ===
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

# === General configuration ===
k_to_p = {18: 0.060, 36: 0.120, 54: 0.180}
epsilons = [0.3, 0.5]
steps = [0, 10_000, 100_000, 300_000]
step_titles = [
    r"Time step $0$",
    r"Time step $1 \times 10^{4}$",
    r"Time step $1 \times 10^{5}$",
    r"Time step $3 \times 10^{5}$"
]

# === Customizable parameters ===
N = 300  # total number of nodes (constant)
max_grado = N - 1
ylim_y = 0.12  # Expected maximum height for P(k)
xlim_x = 180
mostrar_dispersion = False  # Set to True to show ±std shaded areas

# === Visual style ===
bin_edges = np.arange(0, max_grado + 2)  # Integer bins
bin_centers = bin_edges[:-1]
color_map = {0.3: 'blue', 0.5: 'red'}
label_map = {0.3: r"$\varepsilon = 0.3$", 0.5: r"$\varepsilon = 0.5$"}
alfa_mean = 0.6
alfa_std = 0.25

# === File path templates ===
base_template = "results/results_p{:.3f}/simulation_{}/networks/eps_{:.3f}/matrix_eps_{:.3f}_step_{}.txt"
output_dir = "plots_paper/fig4"
os.makedirs(output_dir, exist_ok=True)

# === Loop over different k values ===
for k, p in k_to_p.items():
    fig, axs = plt.subplots(2, 2, figsize=(14, 10))
    axs = axs.flatten()

    for idx, (step, title) in enumerate(zip(steps, step_titles)):
        ax = axs[idx]

        for eps in epsilons:
            all_probs = []

            for sim_id in range(1, 11):  # Simulations 1 to 10
                path = base_template.format(p, sim_id, eps, eps, step)
                if not os.path.exists(path):
                    print(f"⚠ Not found: {path}")
                    continue

                try:
                    adj = pd.read_csv(path, header=None).values
                    adj = (adj > 0).astype(int)
                    degrees = adj.sum(axis=1)

                    counts, _ = np.histogram(degrees, bins=bin_edges)
                    probs = counts / N  # <- This converts to P(k)
                    all_probs.append(probs)

                except Exception as e:
                    print(f"⚠ Error reading {path}: {e}")
                    continue

            if all_probs:
                probs_array = np.array(all_probs)
                mean_probs = probs_array.mean(axis=0)
                std_probs = probs_array.std(axis=0)

                ax.bar(bin_centers, mean_probs, width=1, alpha=alfa_mean,
                       color=color_map[eps], label=label_map[eps],
                       edgecolor='black', linewidth=0.5)

                if mostrar_dispersion:
                    ax.fill_between(bin_centers,
                                    mean_probs - std_probs,
                                    mean_probs + std_probs,
                                    color=color_map[eps], alpha=alfa_std)

        # === Subplot aesthetics ===
        ax.set_title(title, fontsize=18)
        ax.set_xlabel("Node Degree $k$", fontsize=18)
        ax.set_ylabel(r"$P(k)$", fontsize=18)  # <- Updated label
        ax.tick_params(axis='both', labelsize=14)
        ax.grid(True)
        ax.set_axisbelow(True)
        if ylim_y is not None:
            ax.set_ylim(0, ylim_y)
        if xlim_x is not None:
            ax.set_xlim(0, xlim_x)

        # Disable scientific notation on axes
        ax.ticklabel_format(style='plain', axis='y')
        ax.ticklabel_format(style='plain', axis='x')

    # === Global legend ===
    handles = [plt.Line2D([0], [0], color=color_map[eps], lw=5, label=label_map[eps]) for eps in epsilons]
    fig.legend(handles=handles, loc='upper center', bbox_to_anchor=(0.5, 1.02),
               ncol=len(epsilons), fontsize=16, frameon=False)

    # === Save figure ===
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    output_path = os.path.join(output_dir, f"degree_prob_k{k}")
    plt.savefig(output_path + ".png", dpi=300, bbox_inches='tight')
    plt.savefig(output_path + ".pdf", dpi=300, bbox_inches='tight')
    plt.close()

    print(f"✅ Saved: {output_path}.pdf/png")



✅ Saved: plots_paper/fig4\degree_prob_k18.pdf/png
✅ Saved: plots_paper/fig4\degree_prob_k36.pdf/png
✅ Saved: plots_paper/fig4\degree_prob_k54.pdf/png


In [35]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

# === Plot style settings ===
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

# === General configuration ===
k_to_p = {54: 0.180}
epsilons = [0.3, 0.5]
steps = [0, 10_000, 100_000, 300_000]
step_titles = [
    r"Time step $0$",
    r"Time step $1 \times 10^{4}$",
    r"Time step $1 \times 10^{5}$",
    r"Time step $3 \times 10^{5}$"
]

# === Customizable parameters ===
N = 300  # total number of nodes (constant)
max_grado = N - 1
ylim_y = 0.07  # Expected maximum height for P(k)
xlim_x = 180
mostrar_dispersion = False  # Set to True to show ±std shaded areas

# === Visual style ===
bin_edges = np.arange(0, max_grado + 2)  # Integer bins
bin_centers = bin_edges[:-1]
color_map = {0.3: 'blue', 0.5: 'red'}
label_map = {0.3: r"$\varepsilon = 0.3$", 0.5: r"$\varepsilon = 0.5$"}
alfa_mean = 0.6
alfa_std = 0.25

# === File path templates ===
base_template = "results/results_p{:.3f}/simulation_{}/networks/eps_{:.3f}/matrix_eps_{:.3f}_step_{}.txt"
output_dir = "plots_paper/fig4"
os.makedirs(output_dir, exist_ok=True)

# === Loop over different k values ===
for k, p in k_to_p.items():
    fig, axs = plt.subplots(2, 2, figsize=(14, 10))
    axs = axs.flatten()

    for idx, (step, title) in enumerate(zip(steps, step_titles)):
        ax = axs[idx]

        for eps in epsilons:
            all_probs = []

            for sim_id in range(1, 11):  # Simulations 1 to 10
                path = base_template.format(p, sim_id, eps, eps, step)
                if not os.path.exists(path):
                    print(f"⚠ Not found: {path}")
                    continue

                try:
                    adj = pd.read_csv(path, header=None).values
                    adj = (adj > 0).astype(int)
                    degrees = adj.sum(axis=1)

                    counts, _ = np.histogram(degrees, bins=bin_edges)
                    probs = counts / N  # <- This converts to P(k)
                    all_probs.append(probs)

                except Exception as e:
                    print(f"⚠ Error reading {path}: {e}")
                    continue

            if all_probs:
                probs_array = np.array(all_probs)
                mean_probs = probs_array.mean(axis=0)
                std_probs = probs_array.std(axis=0)

                ax.bar(bin_centers, mean_probs, width=1, alpha=alfa_mean,
                       color=color_map[eps], label=label_map[eps],
                       edgecolor='black', linewidth=0.5)

                if mostrar_dispersion:
                    ax.fill_between(bin_centers,
                                    mean_probs - std_probs,
                                    mean_probs + std_probs,
                                    color=color_map[eps], alpha=alfa_std)

        # === Subplot aesthetics ===
        ax.set_title(title, fontsize=18)
        ax.set_xlabel("Node Degree $k$", fontsize=18)
        ax.set_ylabel(r"$P(k)$", fontsize=18)  # <- Updated label
        ax.tick_params(axis='both', labelsize=14)
        ax.grid(True)
        ax.set_axisbelow(True)
        if ylim_y is not None:
            ax.set_ylim(0, ylim_y)
        if xlim_x is not None:
            ax.set_xlim(0, xlim_x)

        # Disable scientific notation on axes
        ax.ticklabel_format(style='plain', axis='y')
        ax.ticklabel_format(style='plain', axis='x')

    # === Global legend ===
    handles = [plt.Line2D([0], [0], color=color_map[eps], lw=5, label=label_map[eps]) for eps in epsilons]
    fig.legend(handles=handles, loc='upper center', bbox_to_anchor=(0.5, 1.02),
               ncol=len(epsilons), fontsize=16, frameon=False)

    # === Save figure ===
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    output_path = os.path.join(output_dir, f"fig4")
    plt.savefig(output_path + ".png", dpi=300, bbox_inches='tight')
    plt.savefig(output_path + ".pdf", dpi=300, bbox_inches='tight')
    plt.close()

    print(f"✅ Saved: {output_path}.pdf/png")



✅ Saved: plots_paper/fig4\fig4.pdf/png
